In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from itertools import zip_longest
import random
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
rating_data = pd.read_csv('ratings.csv')
movie_data = pd.read_csv('movies.csv')
rating_data = rating_data.pivot(index='userId',columns='movieId',values='rating')

In [ ]:
movie_data.info()
rating_data.info()

In [ ]:
def pearson_correlation(u1,rating_data):
      number,std1,std2=0,0,0
      Plist=[]
      u1=u1.dropna()-(u1.mean())
      for n in range(0,len(rating_data)):
        us1,us2=[],[]
        u2=rating_data.iloc[n]
        u2=u2.dropna()-(u2.mean())
        ind=u1.index.intersection(u2.index)
        for i in range(0,len(ind)):
          us1.append(u1.loc[ind[i]])
          us2.append(u2.loc[ind[i]])
        if len(ind)>12:
          number=sum([a*b for a,b in zip(us1,us2)])
          std1=np.sqrt(sum([a*b for a,b in zip(us1,us1)]))
          std2=np.sqrt(sum([a*b for a,b in zip(us2,us2)]))
          sim=(number/(std1*std2))+1
        else :sim=0
        Plist.append(sim)
      return Plist

In [ ]:
def presentList(Plist,i):
    PEARSON = pd.DataFrame({'correlation':Plist})
    a=False
    if i==1:
      u="userID"
      j=11
    else:
      u="chromosomeID"
      j=len(Plist)
      a=True
    PEARSON.insert(1,u, range(1, 1 + len(Plist)))
    PEARSON=PEARSON.sort_values(by='correlation', ascending=a)
    PEARSON=PEARSON.head(j)
    return PEARSON
def topIndex(PEARSON):
    top10=PEARSON.index.to_list()
    top10.pop(0)
    return top10

In [ ]:
def recommendations(u,rating_data,top10):
    u1=rating_data.iloc[u].dropna()
    recom=u1.index
    for t in top10:
        u2=rating_data.iloc[t].dropna()
        recom=recom.union(u2.index)
    recom=recom.difference(u1.index)
    return recom
def findRecommendations2(rating_data,new2,top10,u):
    u1=rating_data.iloc[u].dropna()
    recom=u1.index
    for t in top10:
        u2=rating_data.iloc[t].dropna()
        recom=recom.union(u2.index)
    recom=recom.difference(u1.index)
    return recom

In [ ]:
def fitness_value(pop,Neighbors,recom):
    Fitlist,IndCorList=[],[]
    for i in range(0,len(pop)):
        individual=pop.iloc[i]
        IndCorList=pearson_corr(individual,Neighbors,recom)
        Fitlist.append(np.mean(IndCorList))
    return Fitlist


In [ ]:
def pearson_corr(u1,rating_data,recom):
      num,std1,std2=0,0,0
      Plist=[]
      u1=u1.dropna()-(u1.mean())
      for n in range(0,len(rating_data)):
        us1,us2=[],[]
        u2=rating_data.iloc[n]
        u2=u2.dropna()-(u2.mean())
        ind=recom.intersection(u2.index)
        for i in range(0,len(ind)):
          us1.append(u1.loc[ind[i]])
          us2.append(u2.loc[ind[i]])
        if len(ind)>2:
          num=sum([a*b for a,b in zip(us1,us2)])
          std1=np.sqrt(sum([a*b for a,b in zip(us1,us1)]))
          std2=np.sqrt(sum([a*b for a,b in zip(us2,us2)]))
          sim=(num/(std1*std2))+1
        else :sim=0
        Plist.append(sim)
      return Plist

In [ ]:
def selection(Evaluations,SortedEvaluations,pop):
    Evaluations.sort()
    Evaluations
    N=len(Evaluations)
    TotalFitness=(N+1)*N/2
    IndListIndexes,ProbSelection,CumProb,newList,randomy=[],[],[],[],[]
    for j in range(N):
        x = random.random()
        randomy.append(x)
    for i in range(N):
        if i==0:
          ProbSelection.append(1/TotalFitness)
          CumProb.append(ProbSelection[i])
        else:
          ProbSelection.append((i+1)/TotalFitness)
          CumProb.append(CumProb[i-1]+ProbSelection[i])
    for j in range(N):
      if randomy[j]<CumProb[0]:
          IndListIndexes.append(0)
      else:
        for e in range(N-1):
            if CumProb[e]<randomy[j] and randomy[j]<=CumProb[e+1] :
                IndListIndexes.append(e+1)
    newList=SortedEvaluations.iloc[IndListIndexes].index.to_list()
    NewPop=pd.DataFrame(pop.iloc[newList])
    return NewPop


In [ ]:
def crossover(Selected,Pc,pop_size):
    offspring1,offspring2,offsprings,generator=[],[],[],[]
    for g in range(pop_size):
        y = random.random()
        generator.append(y)
    for i in range(0,len(Selected),2):
        offspring1=[]
        offspring2=[]
        p1=Selected.iloc[i].to_list()
        p2=Selected.iloc[i+1].to_list()
        p = [p1,p2]
        crandomy,randomy=[],[]
        for j in range(len(p1)):
            x = random.randint(0,1)
            randomy.append(x)
            crandomy.append(np.abs(randomy[j]-1))
        if generator[i]<Pc:
          for i in range(len(p[0])):
              offspring1.append(p[randomy[i]][i])
              offspring2.append(p[crandomy[i]][i])
        else:
          for i in range(len(p[0])):
            offspring1.append(p1[i])
            offspring2.append(p2[i])
        offsprings.append(offspring1)
        offsprings.append(offspring2)
    NextGen=pd.DataFrame(offsprings)
    NextGen.index=NextGen.index+1
    NextGen.columns=columns
    NextGen.index.name = 'IndividualID'
    return NextGen

In [ ]:
def mutation(NextGeneration, Pm):
    mutArr= np.random.random(size=(NextGeneration.shape))
    a=np.nonzero(mutArr <0.01)
    NG1=NextGeneration.values
    g1=list(NG1[a])
    for x in range(len(g1)):
      b=random.randint(1,5)
      while True:
          if b!=g1[x]:
            break
          else:
            b=random.randint(1,5)
      g1[x]=b
    NG1[a]=g1
    NewGeneration=pd.DataFrame(NG1)
    NewGeneration.index=NewGeneration.index+1
    NewGeneration.index.name = 'NewGenID'
    NewGeneration.columns=columns
    return NewGeneration

In [ ]:
def rootedMSE(a,u,rating_data,NewGeneration,SortedEvaluations):
    x=NewGeneration.iloc[SortedEvaluations.iloc[-1,1]-1][a].values
    y=rating_data.iloc[u].dropna().values
    return math.sqrt(np.mean((x-y)**2))

In [ ]:
def meanAE(a,u,rating_data,NewGeneration,SortedEvaluations):
    x=NewGeneration.iloc[SortedEvaluations.iloc[-1,1]-1][a].values
    y=rating_data.iloc[u].dropna().values
    return np.mean(abs(x-y))

In [14]:
u=1067
u1=rating_data.iloc[u]
a=rating_data.iloc[u].dropna().index
pop_size,Pc,Pm,maxgen=200,0.9,0.01,300
generation,meanGen,meanBest=1,0,0
fit,temp,divide,temp2,RMSE,MAE,tempRMSE,tempMAE=[],[],[],[],[],[],[],[]
times=10
limit=30
PearList=pearson_correlation(u1,rating_data)
PEARSON=presentList(PearList,1)
NeighborInd=topIndex(PEARSON)
recommendations=findRecommendations2(rating_data,rating_data,NeighborInd,u)
movies=recommendations.to_list()
for i in range(times):
        pop=[np.random.randint(1,6,len(recommendations)) for i in range(pop_size)]
        pop=pd.DataFrame(pop)
        pop.index=pop.index+1
        pop.columns=movies
        pop.index.name = 'IndividualID'
        Neighbors=pd.DataFrame(rating_data.iloc[NeighborInd])
        Evaluations=fitness_value(pop,Neighbors,recommendations)
        SortedEvaluations=presentList(Evaluations,0)
        NewPopulation=selection(Evaluations,SortedEvaluations,pop)
        Selected=NewPopulation.loc[:, movies]
        columns=Selected.columns.values
        NextGeneration=crossover(Selected,Pc,pop_size)
        NewGeneration=mutation(NextGeneration,Pm)
        while generation<maxgen :
              generation+=1
              Evaluations=fitness_value(NewGeneration,Neighbors,recommendations)
              SortedEvaluations=presentList(Evaluations,0)
              NewPopulation=selection(Evaluations,SortedEvaluations,NewGeneration)
              Selected=NewPopulation.loc[:, movies]
              columns=Selected.columns.values
              NextGeneration=crossover(Selected,Pc,pop_size)
              NewGeneration=mutation(NextGeneration,Pm)
              Evaluations=fitness_value(NewGeneration,Neighbors,recommendations)
              SortedEvaluations=presentList(Evaluations,0)
              temp.append(SortedEvaluations.iloc[-1,0])
              print(generation,SortedEvaluations.iloc[-1,0])
              rmse=rootedMSE(a,u,rating_data,NewGeneration,SortedEvaluations)
              mae=meanAE(a,u,rating_data,NewGeneration,SortedEvaluations)
              print('rmse:',rmse,'mae:',mae)
              tempRMSE.append(rmse)
              tempMAE.append(mae)
              if len(temp)>limit :
                if (SortedEvaluations.iloc[-1,0]<temp[-limit]) | ((100*(SortedEvaluations.iloc[-1,0]-temp[-limit])/temp[-limit]) <1e-1):
                    break
        temp2 = np.ones((generation-1), dtype = int)
        meanGen+=generation
        print('temp,generation,temp2',temp,generation,temp2)
        if i==0:
            fit=temp
            divide=temp2
            RMSE=tempRMSE
            MAE=tempMAE
        else:
            fit=[x+y for x,y in zip_longest(fit, temp, fillvalue=0)]
            divide=[x+y for x,y in zip_longest(divide, temp2, fillvalue=0)]
            RMSE=[x+y for x,y in zip_longest(RMSE, tempRMSE, fillvalue=0)]
            MAE=[x+y for x,y in zip_longest(MAE, tempMAE, fillvalue=0)]
        print('qumulative sum of fitnesses:',fit,'\nqumulative counter of generations:',divide,'\nqumulative RMSE:',RMSE,'\nqumulativeMAE:',MAE)
        temp.sort()
        meanBest+=temp[-1]
        print('sortedtemp,qumulbest,qumulgen:',temp,meanBest,meanGen)
        temp,temp2,tempRMSE,tempMAE=[],[],[],[]
        generation=1
meanBest=meanBest/times
meanGen=meanGen/times
fit=[a/b for a,b in zip(fit,divide)]
meanRMSE=[a/b for a,b in zip(RMSE,divide)]
meanMAE=[a/b for a,b in zip(MAE,divide)]
print('fit,meanGen,meanBest',fit,meanGen,meanBest)
